In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.python.solutions import holistic
import tensorflow as tf

In [ ]:
mp_drawing=mp.solutions.drawing_utils   #holistic model
mp_holistic=mp.solutions.holistic       #drawing utilities



In [ ]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #color conversion BGR to RGB
    image.flags.writeable = False                   #Image is no longer writeable
    results = model.process(image)                  # make prediction
    image.flags.writeable = True                    #image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #color conversion again RGB to BGR
    return image,results

def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw right hand connections

def draw_style_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(175,13,13), thickness=1, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(178,16,16), thickness=2, circle_radius=4),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)    #color line
                              )


In [ ]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    # Read feed
    ret,frame=cap.read()
    #make detections
    image, results= mediapipe_detection(frame, holistic)
    print(results)

    # Draw landmarks
    # draw_landmarks(image, results)  #default landmarks in face hand 

    draw_style_landmarks(image, results) #customized landmarks in face and hand
    #show to screen
    cv2.putText(image,"Starting collecting", (120,200), #display message in screen at position x=120 and y=200
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,225,0), 4, cv2.LINE_AA) 
    # Flip the image horizontally
    image = cv2.flip(image, 1)
    cv2.imshow('OPENCV FEED',image)

    if cv2.waitKey(10) & 0xFF==ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()


In [ ]:
#path for exported data, numpy arrays
DATA_PATH = os.path.join('D:\Hand-Gesture-Recognition\Frame_Data')
IMAGES_PATH=os.path.join('D:\Hand-Gesture-Recognition\Frame_COLLECTION')
actions=np.array(['Aausadi','Ambulance','Bathroom','Be Careful','Bleeding','Call','Dhanebad','Doctor','Dont Understand','Eklopan',
                  'Emergency','Firstaid','Good Morning','Happy','Heart_attack','Hello','Help','Hospital','Name','Need','Nice To Meet You',
                  'Oxygen','Pain','Please','Police','Relax','Sign','Slowly','Sorry','What','Yes','You'])

#thirty videos worth of data
no_sequences = 50

#videos are going to be 60 frames in length
sequence_length = 60

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            os.makedirs(os.path.join(IMAGES_PATH, action, str(sequence)))
       
        except:
            pass


In [ ]:
cap=cv2.VideoCapture(0)
#setup mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  
# loop through actions
  for action in actions:
        # loop through sequences
        for sequence in range(no_sequences):
            # loop through number of sequences length
            for frame_num in range(sequence_length):
        # Read feed
                ret,frame=cap.read()
                #make detections
                image, results= mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                # draw_landmarks(image, results)  #default landmarks in face hand 
                draw_style_landmarks(image, results) #customized landmarks in face and hand
                

                if frame_num == 0:
                    cv2.putText(image,"Starting collecting", (120,200), #display message in screen at position x=120 and y=200
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,225,0), 4, cv2.LINE_AA) #using font style 1=size (0,255,0)=color and 4=line width
                    cv2.putText(image,'Collecting frames for {}'.format(action,sequence),(15,12), #same
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1, cv2.LINE_AA) #same
                    # image = cv2.flip(image, 1)
                    cv2.imshow('OPENCV FEED',image)
                    cv2.waitKey(2000) #break for each video
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,sequence),(15,12), #same for if frame_num!=0
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                    # image = cv2.flip(image, 1)
                    cv2.imshow('OPENCV FEED',image)
                    

                #exporting keypoints in respective folders
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path,keypoints)


                
                # Save frames as jpg in respective folders
                img_path = os.path.join(IMAGES_PATH, action, str(sequence), str(frame_num) + '.jpg')
                cv2.imwrite(img_path, image)
                #show to screen
                # print("frame_num:", frame_num)


                key = cv2.waitKey(10)
                if key == ord('q') or key == ord('Q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    raise SystemExit("Program terminated by user")  # Raise SystemExit to exit gracefully

                # if cv2.waitKey(10) & 0xFF==ord('q') or 0xFF==ord('Q'):  #for quiting program
                #     break
                
                
  cap.release()
  cv2.destroyAllWindows()